In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/project/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that

# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME


Mounted at /content/drive
/content/drive/My Drive/cs231n/project


# Initial Setup

In [2]:
# As usual, a bit of setup
!ls
%cd code

import utils
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import RandomUniform, glorot_uniform, constant, identity
from PIL import Image
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

%cd ..
!ls

cnn_vgg_19.ipynb	cnn_vgg_19_old.ipynb  resnet_old_3.ipynb
cnn_vgg_19_old_2.ipynb	code		      resnet_old_4.ipynb
cnn_vgg_19_old_3.ipynb	resent_old_2.ipynb    resnet_old_5.ipynb
cnn_vgg_19_old_4.ipynb	resnet.ipynb	      resnet_old.ipynb
/content/drive/My Drive/cs231n/project/code
/content/drive/My Drive/cs231n/project
cnn_vgg_19.ipynb	cnn_vgg_19_old.ipynb  resnet_old_3.ipynb
cnn_vgg_19_old_2.ipynb	code		      resnet_old_4.ipynb
cnn_vgg_19_old_3.ipynb	resent_old_2.ipynb    resnet_old_5.ipynb
cnn_vgg_19_old_4.ipynb	resnet.ipynb	      resnet_old.ipynb


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun  2 06:11:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


# Identity Block

In [5]:
def identity_block(X, f, filters, training=True, initializer=RandomUniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ### START CODE HERE
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) 
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X) 
    ### END CODE HERE

    return X

# Convolutional Block

In [6]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ### START CODE HERE
    
    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training) 
    X = Activation('relu')(X) 

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)  
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)
    
    ### END CODE HERE

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

# Create Resnet-152

In [7]:
def ResNet152(input_shape = (224 ,224, 3), classes = 7):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters = [128, 128, 512], s=2) 
    for i in range(7):
      X = identity_block(X, 3, [128, 128, 512]) 
      #X = identity_block(X, 3, [128, 128, 512])
      #X = identity_block(X, 3, [128, 128, 512]) 
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters = [256, 256, 1024], s=2) 
    for i in range(35):
      X = identity_block(X, 3, [256, 256, 1024]) 
      #X = identity_block(X, 3, [256, 256, 1024]) 
      #X = identity_block(X, 3, [256, 256, 1024]) 
      #X = identity_block(X, 3, [256, 256, 1024]) 
      #X = identity_block(X, 3, [256, 256, 1024]) 

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters = [512, 512, 2048], s=2) 
    X = identity_block(X, 3, [512, 512, 2048]) 
    X = identity_block(X, 3, [512, 512, 2048]) 

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

# Testing the mean values

In [ ]:
img1 = np.ones((1,224,224,3))
img2 = np.zeros((1,224,224,3))
img3 = np.ones((1,224,224,3))
img4 = np.ones((1,224,224,3))
values = np.vstack([img1, img2, img3, img4])
N = values.shape[0]
means = np.sum(values, axis=0) / N
mean = np.mean(values)
print((img2 - means)[0].shape)
print(mean)

# Load the dataset

In [8]:
# Loading the data 
!ls
%cd code
orig_x, orig_y = utils.read_data()
orig_y = utils.classification(orig_y, orig_y)
orig_y = utils.one_hot_labels(orig_y)
orig_x = orig_x / 255.
#orig_x -= 0.5
#orig_x *= 2
# orig_x = orig_x-np.mean(orig_x)
# Create the training sample
train_x, val_test_x, train_y, val_test_y = train_test_split(orig_x, orig_y, test_size=0.3, random_state=1)
# Split the remaining observations into validation and test
val_x, test_x, val_y, test_y = train_test_split(val_test_x, val_test_y, test_size=0.33, random_state=1)
%cd ..

cnn_vgg_19.ipynb	cnn_vgg_19_old.ipynb  resnet_old_3.ipynb
cnn_vgg_19_old_2.ipynb	code		      resnet_old_4.ipynb
cnn_vgg_19_old_3.ipynb	resent_old_2.ipynb    resnet_old_5.ipynb
cnn_vgg_19_old_4.ipynb	resnet.ipynb	      resnet_old.ipynb
/content/drive/MyDrive/cs231n/project/code
/content/drive/My Drive/cs231n/project/code/images
(1, 500, 500, 3)
(2, 500, 500, 3)
(3, 500, 500, 3)
(4, 500, 500, 3)
(5, 500, 500, 3)
(6, 500, 500, 3)
(7, 500, 500, 3)
(8, 500, 500, 3)
(9, 500, 500, 3)
(10, 500, 500, 3)
(11, 500, 500, 3)
(12, 500, 500, 3)
(13, 500, 500, 3)
(14, 500, 500, 3)
(15, 500, 500, 3)
(16, 500, 500, 3)
(17, 500, 500, 3)
(18, 500, 500, 3)
(19, 500, 500, 3)
(20, 500, 500, 3)
(21, 500, 500, 3)
(22, 500, 500, 3)
(23, 500, 500, 3)
(24, 500, 500, 3)
(25, 500, 500, 3)
(26, 500, 500, 3)
(27, 500, 500, 3)
(28, 500, 500, 3)
(29, 500, 500, 3)
(30, 500, 500, 3)
(31, 500, 500, 3)
(32, 500, 500, 3)
(33, 500, 500, 3)
(34, 500, 500, 3)
(35, 500, 500, 3)
(36, 500, 500, 3)
(37, 500, 500, 3)
(38, 500, 500,

# Check the dataset

In [9]:
# Example of an image from the dataset
#index = 270
#plt.imshow(orig_x[index])
#print(orig_x[index])

print("number of training examples = " + str(train_x.shape[0]))
print("number of validation examples = " + str(val_x.shape[0]))
print("X_train shape: " + str(train_x.shape))
print("Y_train shape: " + str(train_y.shape))
print("X_val shape: " + str(val_x.shape))
print("Y_val shape: " + str(val_y.shape))
print("X_test shape: " + str(test_x.shape))
print("Y_test shape: " + str(test_y.shape))

number of training examples = 637
number of validation examples = 182
X_train shape: (637, 500, 500, 3)
Y_train shape: (637, 7)
X_val shape: (182, 500, 500, 3)
Y_val shape: (182, 7)
X_test shape: (91, 500, 500, 3)
Y_test shape: (91, 7)


# Create the model - using tensorflow

In [10]:
from tensorflow.python.eager.context import set_log_device_placement
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa

# Create the model
model = ResNet152(input_shape=(224 , 224, 3), classes=7)
# Establishes an exponential decay
lr_schedule = tf.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00025,
    decay_steps=640,
    decay_rate=0.9
)
# Define an Adam optimizer
#adam = tfa.optimizers.AdamW(weight_decay=5e-4, learning_rate=lr_schedule)
adam = tf.optimizers.Adam(learning_rate=lr_schedule)
#sgd = tf.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 13.8 MB/s 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )   

# Train the model

In [ ]:
# Create the image generator
datagen = ImageDataGenerator(featurewise_std_normalization=True)
train_batches = datagen.flow(train_x, train_y, batch_size=64)
train_crops = utils.crop_generator(train_batches, 224)
val_batches = datagen.flow(val_x, val_y, batch_size=64)
val_crops = utils.crop_generator(val_batches, 224)
test_batches = datagen.flow(test_x, test_y, batch_size=1, shuffle=False)
test_crops = utils.crop_generator(test_batches, 224)

history = model.fit_generator(train_crops, epochs=1000, steps_per_epoch=64,
                                       validation_data=val_crops, validation_steps=64)

print(history.history)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_s

Epoch 1/1000
64/64 [==============================] - 104s 1s/step - loss: 3.4405 - accuracy: 0.2513 - val_loss: 2.7798 - val_accuracy: 0.2491
Epoch 2/1000
64/64 [==============================] - 65s 1s/step - loss: 2.1851 - accuracy: 0.3215 - val_loss: 2.2829 - val_accuracy: 0.3527
Epoch 3/1000
64/64 [==============================] - 65s 1s/step - loss: 2.0140 - accuracy: 0.3806 - val_loss: 2.1882 - val_accuracy: 0.3415
Epoch 4/1000
64/64 [==============================] - 65s 1s/step - loss: 2.1209 - accuracy: 0.3975 - val_loss: 1.9905 - val_accuracy: 0.3515
Epoch 5/1000
64/64 [==============================] - 65s 1s/step - loss: 1.9402 - accuracy: 0.4191 - val_loss: 1.9405 - val_accuracy: 0.3780
Epoch 6/1000
64/64 [==============================] - 65s 1s/step - loss: 1.6681 - accuracy: 0.4532 - val_loss: 1.9167 - val_accuracy: 0.3909
Epoch 7/1000
64/64 [==============================] - 65s 1s/step - loss: 1.6858 - accuracy: 0.4566 - val_loss: 1.8015 - val_accuracy: 0.3832
Epoch

# Plot history

In [ ]:
df_loss_acc = pd.DataFrame(history.history)
df_loss = df_loss_acc[['loss', 'val_loss']]
df_acc = df_loss_acc[['accuracy', 'val_accuracy']]
df_loss.plot(title='Model loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')
df_acc.plot(title='Model Accuracy', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')
plt.show()

# Test the model

In [ ]:
temp_x, y = [], []
x = np.empty((1000, 224, 224, 3))
for i in range(1000):
    a, b = test_crops.__next__()
    temp_x.extend(a)
    x[i,] = a
    y.extend(b)
print(x.shape)
Pred_y = model.predict(x)
pred_y = np.argmax(Pred_y, axis=1)
Test_y = np.array(y)
test_y = np.argmax(Test_y, axis=1)
print(test_y)
print('pred_y')
print(pred_y)
# Calculate accuracy, precision, recall and confusion matrix
print('Test Accuracy: ', accuracy_score(test_y, pred_y))
print('Test Precision: ', precision_score(test_y, pred_y, zero_division=0, average='weighted'))
print('Test Recall: ', recall_score(test_y, pred_y, average='weighted'))
print('Test f1', f1_score(test_y, pred_y, zero_division=0, average='weighted', labels=np.unique(pred_y)))


# Confusion Matrix

In [ ]:
cm = confusion_matrix(test_y, pred_y)
print('Confusion Matrix')
print(cm)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm)
cmd.plot()
plt.show()

# Calculate the scores

In [ ]:
scores = model.evaluate(test_crops, steps=1000)
print(scores)